In [2]:
import pandas as pd
raw_path = "data/rawdata/function.json"
raw = pd.read_json(raw_path)
raw

,project,commit_id,target,func
0,FFmpeg,973b1a6b9070e2bf17d17568cbaf4043ce931f51,0,static av_cold int vdadec_init(AVCodecContext ...
1,FFmpeg,321b2a9ded0468670b7678b7c098886930ae16b2,0,static int transcode(AVFormatContext **output_...
2,FFmpeg,5d5de3eba4c7890c2e8077f5b4ae569671d11cf8,0,"static void v4l2_free_buffer(void *opaque, uin..."
3,FFmpeg,32bf6550cb9cc9f487a6722fe2bfc272a93c1065,0,"int ff_get_wav_header(AVFormatContext *s, AVIO..."
4,FFmpeg,57d77b3963ce1023eaf5ada8cba58b9379405cc8,0,"int av_opencl_buffer_write(cl_mem dst_cl_buf, ..."
...,...,...,...,...
27313,qemu,1ea879e5580f63414693655fcf0328559cdce138,0,"static int no_init_in (HWVoiceIn *hw, audsetti..."
27314,qemu,f74990a5d019751c545e9800a3376b6336e77d38,0,"uint32_t HELPER(stfle)(CPUS390XState *env, uin..."
27315,qemu,a89f364ae8740dfc31b321eed9ee454e996dc3c1,0,"static void pxa2xx_fir_write(void *opaque, hwa..."
27316,qemu,39fb730aed8c5f7b0058845cb9feac0d4b177985,0,"static void disas_thumb_insn(CPUARMState *env,..."


In [12]:
raw = raw.reset_index().rename(columns={'index': 'idx'})
raw

,idx,project,commit_id,target,func
0,0,FFmpeg,973b1a6b9070e2bf17d17568cbaf4043ce931f51,0,static av_cold int vdadec_init(AVCodecContext ...
1,1,FFmpeg,321b2a9ded0468670b7678b7c098886930ae16b2,0,static int transcode(AVFormatContext **output_...
2,2,FFmpeg,5d5de3eba4c7890c2e8077f5b4ae569671d11cf8,0,"static void v4l2_free_buffer(void *opaque, uin..."
3,3,FFmpeg,32bf6550cb9cc9f487a6722fe2bfc272a93c1065,0,"int ff_get_wav_header(AVFormatContext *s, AVIO..."
4,4,FFmpeg,57d77b3963ce1023eaf5ada8cba58b9379405cc8,0,"int av_opencl_buffer_write(cl_mem dst_cl_buf, ..."
...,...,...,...,...,...
27313,27313,qemu,1ea879e5580f63414693655fcf0328559cdce138,0,"static int no_init_in (HWVoiceIn *hw, audsetti..."
27314,27314,qemu,f74990a5d019751c545e9800a3376b6336e77d38,0,"uint32_t HELPER(stfle)(CPUS390XState *env, uin..."
27315,27315,qemu,a89f364ae8740dfc31b321eed9ee454e996dc3c1,0,"static void pxa2xx_fir_write(void *opaque, hwa..."
27316,27316,qemu,39fb730aed8c5f7b0058845cb9feac0d4b177985,0,"static void disas_thumb_insn(CPUARMState *env,..."


In [14]:
import warnings
warnings.filterwarnings('ignore')
import clang.cindex
import clang.enumerations

clang.cindex.Config.set_library_path("/usr/lib/x86_64-linux-gnu")
clang.cindex.Config.set_library_file('/usr/lib/x86_64-linux-gnu/libclang-6.0.so.1')

import re 
import os

keywords = ["alignas", "alignof", "and", "and_eq", "asm", "atomic_cancel", "atomic_commit", 
            "atomic_noexcept", "auto", "bitand", "bitor", "bool", "break", "case", "catch", 
            "char", "char8_t", "char16_t", "char32_t", "class", "compl", "concept", "const", 
            "consteval", "constexpr", "constinit", "const_cast", "continue", "co_await", 
            "co_return", "co_yield", "decltype", "default", "delete", "do", "double", "dynamic_cast", 
            "else", "enum", "explicit", "export", "extern", "false", "float", "for", "friend", "goto", 
            "if", "inline", "int", "long", "mutable", "namespace", "new", "noexcept", "not", "not_eq", 
            "nullptr", "operator", "or", "or_eq", "private", "protected", "public", "reflexpr", 
            "register", "reinterpret_cast", "requires", "return", "short", "signed", "sizeof", "static", 
            "static_assert", "static_cast", "struct", "switch", "synchronized", "template", "this", 
            "thread_local", "throw", "true", "try", "typedef", "typeid", "typename", "union", "unsigned", 
            "using", "virtual", "void", "volatile", "wchar_t", "while", "xor", "xor_eq", "NULL"]

class Tokenizer:
    # creates the object, does the inital parse
    def __init__(self, path, tokenizer_type='original'):
        self.index = clang.cindex.Index.create()
        self.tu = self.index.parse(path)
        self.path = self.extract_path(path)
        self.symbol_table = {}
        self.symbol_count = 1
        self.tokenizer_type = tokenizer_type

    # To output for split_functions, must have same path up to last two folders
    def extract_path(self, path):
        return "".join(path.split("/")[:-2])

    
    def full_tokenize_cursor(self, cursor):
        tokens = cursor.get_tokens()
        result = []
        for token in tokens:
            if token.kind.name == "COMMENT":
                continue
            if token.kind.name == "LITERAL":
                result += self.process_literal(token)
                continue
            if token.kind.name == "IDENTIFIER":
                if token.spelling in keywords:
                    result += [token.spelling]
                else:
                    result += ["ID"]
                continue
            result += [token.spelling]
        return result

    def full_tokenize(self):
        cursor = self.tu.cursor
        return self.full_tokenize_cursor(cursor)

    def process_literal(self, literal):
        cursor_kind = clang.cindex.CursorKind
        kind = literal.cursor.kind
        if kind == cursor_kind.INTEGER_LITERAL:
            return literal.spelling
        if kind == cursor_kind.FLOATING_LITERAL:
            return literal.spelling
        if kind == cursor_kind.IMAGINARY_LITERAL:
            return ["NUM"]       
        if kind == cursor_kind.STRING_LITERAL:
            return ["STRING"]
        sp = literal.spelling
        if re.match('[0-9]+', sp) is not None:
            return sp
        return ["LITERAL"]

    def split_functions(self, method_only):
        # results = []
        cursor_kind = clang.cindex.CursorKind
        cursor = self.tu.cursor
        for c in cursor.get_children():
            filename = c.location.file.name if c.location.file != None else "NONE"
            extracted_path = self.extract_path(filename)

            if (c.kind == cursor_kind.CXX_METHOD or (method_only == False and c.kind == cursor_kind.FUNCTION_DECL)) and extracted_path == self.path:
                name = c.spelling
                tokens = self.full_tokenize_cursor(c)
                filename = filename.split("/")[-1]
                return tokens
                # results += [tokens]
                # break;
        # return results
        return None
    

def tokenize(file_text, filename):
    try:
        c_file = open(f'/tmp/{filename}.c', 'w')
        c_file.write(file_text)
        c_file.close()
        tok = Tokenizer(f'/tmp/{filename}.c')
        results = tok.split_functions(False)
        # print(results)
        # return ' '.join(results[0])
        os.remove(f'/tmp/{filename}.c')
        return results
    except:
        return None

In [16]:
from tqdm import tqdm
tqdm.pandas(desc="Tokenizer")
raw['tokenized'] = raw.progress_apply(lambda row: tokenize(row['func'], row['idx']), axis=1)
raw

Tokenizer: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27318/27318 [03:12<00:00, 141.86it/s]


,idx,project,commit_id,target,func,tokenized
0,0,FFmpeg,973b1a6b9070e2bf17d17568cbaf4043ce931f51,0,static av_cold int vdadec_init(AVCodecContext ...,None
1,1,FFmpeg,321b2a9ded0468670b7678b7c098886930ae16b2,0,static int transcode(AVFormatContext **output_...,"[static, int, ID, (, ID, *, *, ID, ,, int, ID,..."
2,2,FFmpeg,5d5de3eba4c7890c2e8077f5b4ae569671d11cf8,0,"static void v4l2_free_buffer(void *opaque, uin...","[static, void, ID, (, void, *, ID, ,, ID, *, I..."
3,3,FFmpeg,32bf6550cb9cc9f487a6722fe2bfc272a93c1065,0,"int ff_get_wav_header(AVFormatContext *s, AVIO...","[int, ID, (, ID, *, ID, ,, ID, *, ID, ,, ID, *..."
4,4,FFmpeg,57d77b3963ce1023eaf5ada8cba58b9379405cc8,0,"int av_opencl_buffer_write(cl_mem dst_cl_buf, ...","[int, ID, (, ID, ID, ,, ID, *, ID, ,, ID, ID, ..."
...,...,...,...,...,...,...
27313,27313,qemu,1ea879e5580f63414693655fcf0328559cdce138,0,"static int no_init_in (HWVoiceIn *hw, audsetti...","[static, int, ID, (, ID, *, ID, ,, ID, *, ID, ..."
27314,27314,qemu,f74990a5d019751c545e9800a3376b6336e77d38,0,"uint32_t HELPER(stfle)(CPUS390XState *env, uin...","[ID, ID, (, ID, ), (, ID, *, ID, ,, ID, ID, ),..."
27315,27315,qemu,a89f364ae8740dfc31b321eed9ee454e996dc3c1,0,"static void pxa2xx_fir_write(void *opaque, hwa...","[static, void, ID, (, void, *, ID, ,, ID, ID, ..."
27316,27316,qemu,39fb730aed8c5f7b0058845cb9feac0d4b177985,0,"static void disas_thumb_insn(CPUARMState *env,...","[static, void, ID, (, ID, *, ID, ,, ID, *, ID,..."


In [20]:
raw[raw.tokenized.map(lambda v: True if v is None else False)]

,idx,project,commit_id,target,func,tokenized
0,0,FFmpeg,973b1a6b9070e2bf17d17568cbaf4043ce931f51,0,static av_cold int vdadec_init(AVCodecContext ...,None
19,19,FFmpeg,2df0c32ea12ddfa72ba88309812bfb13b674130f,0,av_cold void ff_af_queue_init(AVCodecContext *...,None
26,26,FFmpeg,16c831851384ab59e73579fdd9913fbff3c0284a,1,static av_cold int iv_alloc_frames(Indeo3Decod...,None
29,29,FFmpeg,f61bece684d9685b07895508e6c1c733b5564ccf,0,av_cold void ff_float_dsp_init_ppc(AVFloatDSPC...,None
31,31,FFmpeg,3b199d29cd597a3518136d78860e172060b9e83d,0,static av_cold int rpza_decode_init(AVCodecCon...,None
...,...,...,...,...,...,...
27180,27180,qemu,1f01e50b8330c24714ddca5841fdbb703076b121,0,static int coroutine_fn qed_aio_write_alloc(QE...,None
27199,27199,FFmpeg,f6b7f72461673e4d398b1edf9ed2a7fe70d99c47,0,static void av_always_inline filter_mb_edgeh( ...,None
27206,27206,qemu,d9bce9d99f4656ae0b0127f7472db9067b8f84ab,1,void OPPROTO op_srli_T1 (void)\n\n{\n\n T1 ...,None
27296,27296,FFmpeg,d6604b29ef544793479d7fb4e05ef6622bb3e534,0,static av_cold int libschroedinger_encode_init...,None


In [24]:
true_data = raw[~raw.tokenized.map(lambda v: True if v is None else False)]

In [27]:
true_data.tokenized.map(len).min()

7

In [29]:
root = 'data'
raw_data = os.path.join(root, 'rawdata')
files = ['train','valid', 'test']
for k in files:
    with open(os.path.join(raw_data, f'{k}.txt'), 'r') as fp:
        slices = [int(v.strip()) for v in fp.readlines()]
        temp = raw.iloc[slices]
        temp = temp[~temp.tokenized.map(lambda v: True if v is None else False)]
        temp = temp.reset_index(drop=True)
        temp.to_json(os.path.join(root, f'{k}.json'), orient='records')
        print(f'save {root}/{k}.json')
print('Done')

save data/train.json
save data/valid.json
save data/test.json
Done


In [31]:
train_data = pd.read_json('data/train.json')
train_data

,idx,project,commit_id,target,func,tokenized
0,6892,qemu,9d7a4c6690ef9962a3b20034f65008f1ea15c1d6,1,void *g_try_malloc0(size_t n_bytes)\n\n{\n\n ...,"[void, *, ID, (, ID, ID, ), {, ID, (, ID, ), ;..."
1,180,qemu,d9f62dde1303286b24ac8ce88be27e2b9b9c5f46,0,"static void prop_get_fdt(Object *obj, Visitor ...","[static, void, ID, (, ID, *, ID, ,, ID, *, ID,..."
2,6554,qemu,baf905e580ab9c8eaf228822c4a7b257493b4998,1,"static void test_blk_read(BlockBackend *blk, l...","[static, void, ID, (, ID, *, ID, ,, long, ID, ..."
3,17061,qemu,a8170e5e97ad17ca169c64ba87ae2f53850dab4c,0,"static void dchip_write(void *opaque, target_p...","[static, void, ID, (, void, *, ID, ,, ID, ID, ..."
4,18027,FFmpeg,f1cdc01e7208f9e05cd92127003b3b494ec10cfb,0,static OutputStream *new_output_stream(Options...,"[static, ID, *, ID, (, ID, *, ID, ,, ID, *, ID..."
...,...,...,...,...,...,...
20805,5697,qemu,37654d9e6af84003982f8b9a5d59a4aef28e0a79,0,static inline void _t_gen_mov_env_TN(int offse...,"[static, inline, void, ID, (, int, ID, ,, ID, ..."
20806,20432,FFmpeg,6a6bc43f5f79587b8936334cc0b3a6616f4807ac,0,static int dxtory_decode_v2_410(AVCodecContext...,"[static, int, ID, (, ID, *, ID, ,, ID, *, ID, ..."
20807,1928,FFmpeg,72ec043af4510723c53c729a67be482a14b7c7f3,1,"static int nprobe(AVFormatContext *s, uint8_t ...","[static, int, ID, (, ID, *, ID, ,, ID, *, ID, ..."
20808,23958,qemu,b14ef7c9ab41ea824c3ccadb070ad95567cca84e,1,void do_unassigned_access(target_phys_addr_t a...,"[void, ID, (, ID, ID, ,, int, ID, ,, int, ID, ..."


In [68]:
tokens: pd.Series = train_data.tokenized.progress_apply(pd.Series).stack()
tokens = tokens.value_counts()
tokens

Tokenizer: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20810/20810 [00:50<00:00, 410.85it/s]


ID                                                              2285234
;                                                                579778
)                                                                536609
(                                                                536572
,                                                                438735
                                                                 ...   
're running on a native PCIe machine type, like Q35, we need          1
't                                                                    1
're attached to express buses in the VM.                              1
't belong on the root complex.  Windows                               1
't support LNK/LNK2 capabilities, so make them up.                    1
Length: 158, dtype: int64

In [74]:
tokens = tokens[tokens > 1].index.values
tokens

array(['ID', ';', ')', '(', ',', '->', '=', '0', '{', '}', '[', ']', '*',
       '1', 'if', '&', 'LITERAL', '.', ':', 'int', '2', '+', '-',
       'return', 'case', 'x', '<', '==', 'break', 'else', '4', '3', '!',
       '++', '8', '6', 'f', '&&', 'NULL', 'for', '#', 'static', 'void',
       '5', '!=', 'const', '>>', 'goto', '+=', '<<', '||', 'sizeof', '>',
       '7', '|', 'char', 'STRING', 'F', '/', '?', '>=', '9', 'unsigned',
       'switch', 'struct', '|=', 'default', 'while', '~', '<=', '%', '-=',
       'c', 'L', '&=', 'true', 'e', '--', 'continue', 'false', 'bool',
       'float', 'long', 'a', 'd', 'b', 'U', 'double', 'inline', '...',
       '^', 'C', 'volatile', 'E', '*=', 'do', 'short', '>>=', '##',
       'enum', '__func__', 'l', '<<=', '__asm__', 'u', 'B', 'signed', 'A',
       'asm', 'D', '^=', '/=', '__FUNCTION__', 'register', '__asm',
       '__volatile', 'class', 'new', 'typename', 'private', 'T', '%=',
       'export', '__attribute__', 'union', 'and', 'or', 'virtual', 'x

In [43]:
train_data.iloc[15473]

idx                                                      20863
project                                                   qemu
commit_id             47985727e383eee191a39cc007541c17cd8680ee
target                                                       1
func         static int vfio_setup_pcie_cap(VFIOPCIDevice *...
tokenized    [static, int, ID, (, ID, *, ID, ,, int, ID, ,,...
Name: 15473, dtype: object

In [78]:
import torch
from torchtext.vocab import vocab
from collections import OrderedDict
v1 = vocab(OrderedDict([(token, 1) for token in tokens]))
v1.insert_token('<pad>', 0)
v1.insert_token('<unk>', 1)
v1.set_default_index(v1['<unk>'])
len(v1)

151

In [81]:
v1.lookup_tokens(v1(train_data.iloc[0].tokenized))

['void',
 '*',
 'ID',
 '(',
 'ID',
 'ID',
 ')',
 '{',
 'ID',
 '(',
 'ID',
 ')',
 ';',
 'return',
 'ID',
 '(',
 '1',
 ',',
 'ID',
 '==',
 '0',
 '?',
 '1',
 ':',
 'ID',
 ')',
 ';',
 '}']

In [95]:
v1['<pad>']

0

In [92]:
v1.lookup_tokens(v1(train_data.iloc[15473].tokenized))

['static',
 'int',
 'ID',
 '(',
 'ID',
 '*',
 'ID',
 ',',
 'int',
 'ID',
 ',',
 'ID',
 'ID',
 ',',
 'ID',
 '*',
 '*',
 'ID',
 ')',
 '{',
 'ID',
 'ID',
 ';',
 'ID',
 'ID',
 ';',
 'ID',
 '=',
 'ID',
 '(',
 'ID',
 '->',
 'ID',
 '.',
 'ID',
 '+',
 'ID',
 '+',
 'ID',
 ')',
 ';',
 'ID',
 '=',
 '(',
 'ID',
 '&',
 'ID',
 ')',
 '>>',
 '4',
 ';',
 'if',
 '(',
 'ID',
 '!=',
 'ID',
 '&&',
 'ID',
 '!=',
 'ID',
 '&&',
 'ID',
 '!=',
 'ID',
 ')',
 '{',
 'ID',
 '(',
 'ID',
 ',',
 'LITERAL',
 'LITERAL',
 ',',
 'ID',
 ')',
 ';',
 'return',
 '-',
 'ID',
 ';',
 'if',
 '(',
 '!',
 'ID',
 '(',
 'ID',
 '->',
 'ID',
 '.',
 'ID',
 ')',
 ')',
 '{',
 'ID',
 '*',
 'ID',
 '=',
 'ID',
 '->',
 'ID',
 '.',
 'ID',
 ';',
 'ID',
 '*',
 'ID',
 ';',
 '*',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',
 '*',
 'ID',
 '-',
 'ID',
 'ID',
 'ID',
 '-',
 'ID',
 'ID',
 '.',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',
 '*',
 'ID',
 'ID',
 'ID',
 'ID',
 '<unk>',
 '*',
 'ID',
 '<unk>',
 '*',
 'ID',
 'ID',
 'ID',
 'ID',
 'ID',


In [97]:
import pickle
with open('data/vocab', 'wb+') as fp:
    pickle.dump(v1, fp)

In [98]:
with open('data/vocab', 'rb') as fp:
    v2 = pickle.load(fp)
len(v2)

151

In [99]:
v2['<pad>']

0